In [ ]:
%matplotlib inline
import numpy as np
import pandas as pd
import os

### Create the INSTANCES for Whole Agencies

In [ ]:
agency_number = 1
if type(agency_number) == int:
    agency_number = str(agency_number)

In [ ]:
Positions_Raw_Loc = '../Data/Position_Data_Records/'
Agency_Info_Loc = '../Agency-Info/'

In [ ]:
directory = os.listdir(Positions_Raw_Loc)
try:
    directory.index('.DS_Store')
    directory.remove('.DS_Store')
except:
    pass
routes_file_loc = Agency_Info_Loc + agency_number + '_gtfs/routes.txt'

In [ ]:
Timecalls = [i.replace('Positions_' + agency_number + '_','') for i in directory]
Timecalls = [i.replace('.csv','') for i in Timecalls]

In [ ]:
positions_file_locs =  [Positions_Raw_Loc + p for p in directory]

In [ ]:
positions_file_locs_for_agency = [loc for loc in positions_file_locs if loc.find("_" + agency_number + "_")]

In [ ]:
databases = [pd.read_csv(loc).dropna() for loc in positions_file_locs_for_agency]
routes_data = pd.read_csv(routes_file_loc, usecols=['agency_id', 'route_id', 'route_short_name'])

In [ ]:
new_databases = []
for db,t in zip(databases,Timecalls):
    em=[]
    em=[t]*len(db)
    trial = db.reset_index()
    trial.drop('index',axis=1,inplace=True)
    trial['Timecall'] = em
    new_databases.append(trial)

In [ ]:
try:
    INSTANCES = pd.concat(new_databases)
except:
    print "Cannot Create INSTANCES for this Agency"
    INSTANCES = pd.DataFrame([])

In [ ]:
routes_data['route_id'] = routes_data['agency_id'].astype(str) + "_" + routes_data['route_id'].astype(str) 
routes_data['route_short_name'] = routes_data['agency_id'].astype(str) + "_" + routes_data['route_short_name'].astype(str) 

In [ ]:
routes_data.columns = ['Agency_Id', 'Route_Id', 'Route_Short_Name']

In [ ]:
INSTANCES = INSTANCES.merge(routes_data[['Route_Id', 'Route_Short_Name']], how="left", on='Route_Id')

In [ ]:
INSTANCES.to_csv('../Data/INSTANCES_Databases/Whole_Agency/INSTANCES_By_Agency_' + agency_number + '_Whole.txt', sep=',', na_rep='NA', index=False, mode = 'wb')

### Create INSTANCES for One Route at Selected Agency

In [ ]:
route_number = 2
if type(route_number) == int:
    route_number = str(route_number)

In [ ]:
INSTANCES_ROUTE = INSTANCES[INSTANCES['Route_Short_Name']==agency_number + "_" + route_number]

In [ ]:
if os.path.isdir('../Data/INSTANCES_Databases/Route_By_Agency/Agency_' + agency_number + '/'):
    pass
else:
    os.mkdir('../Data/INSTANCES_Databases/Route_By_Agency/Agency_' + agency_number + '/')

In [ ]:
INSTANCES_ROUTE.to_csv('../Data/INSTANCES_Databases/Route_By_Agency/Agency_' + agency_number + '/' + 'INSTANCES_Agency_' + agency_number + '_Route_' + route_number + '.txt', sep=',', na_rep='NA', index=False, mode = 'wb')

### Create INSTANCES_By_Route for All Listed Routes in INSTANCES

In [ ]:
List_of_Routes_In_Agency_INSTANCES = INSTANCES['Route_Short_Name'].unique()

In [ ]:
if os.path.isdir('../Data/INSTANCES_Databases/Route_By_Agency/Agency_' + agency_number + '/'):
    pass
else:
    os.mkdir('../Data/INSTANCES_Databases/Route_By_Agency/Agency_' + agency_number + '/')
for Route in List_of_Routes_In_Agency_INSTANCES:
    Route = Route.replace(agency_number+"_",'')
    INSTANCES_ROUTE = INSTANCES[INSTANCES['Route_Short_Name']==agency_number + "_" + Route]
    INSTANCES_ROUTE.to_csv('../Data/INSTANCES_Databases/Route_By_Agency/Agency_' + agency_number + '/' + 'INSTANCES_Agency_' + agency_number + '_Route_' + Route + '.txt', sep=',', na_rep='NA', index=False, mode = 'wb')